In [33]:
!pip install openai
import openai
import os
from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    pca_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
    chart_from_components_3D
)

In [35]:
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY 

In [14]:
menu_database = {
    "Restaurant A": {
        "Appetizer": {
            "Bruschetta with Tomato and Basil": ["Baguette", "Tomato", "Basil", "Garlic"],
            "Garlic Butter Shrimp": ["Shrimp", "Garlic", "Butter", "Parsley"],
            "Caprese Salad": ["Tomato", "Mozzarella", "Basil", "Balsamic Glaze"]
        },
        "Salad & Soup": {
            "Caesar Salad": ["Romaine Lettuce", "Croutons", "Parmesan Cheese", "Caesar Dressing"],
            "Tomato Bisque": ["Tomatoes", "Cream", "Onion", "Garlic"],
            "French Onion Soup": ["Onions", "Beef Broth", "Cheese", "Bread"]
        },
        "Main": {
            "Grilled Salmon with Lemon Butter": ["Salmon", "Lemon", "Butter", "Asparagus"],
            "Pasta Alfredo with Chicken": ["Pasta", "Chicken", "Cream Sauce", "Parmesan Cheese"],
            "Filet Mignon with Mushroom Sauce": ["Beef", "Mushroom Sauce", "Potatoes", "Green Beans"]
        },
        "Dessert": {
            "Tiramisu": ["Ladyfingers", "Coffee", "Mascarpone Cheese", "Cocoa Powder"],
            "Chocolate Fondue": ["Chocolate", "Strawberries", "Banana", "Marshmallows"],
            "New York Cheesecake": ["Cream Cheese", "Graham Cracker Crust", "Sour Cream", "Raspberry Sauce"]
        }
    },
    "Restaurant B": {
        "Appetizer": {
            "Spinach and Artichoke Dip": ["Spinach", "Artichoke Hearts", "Cream Cheese", "Parmesan"],
            "Mozzarella Sticks": ["Mozzarella Cheese", "Breadcrumbs", "Marinara Sauce"],
            "Stuffed Mushrooms": ["Mushrooms", "Sausage", "Cream Cheese", "Garlic"]
        },
        "Salad & Soup": {
            "Greek Salad": ["Romaine Lettuce", "Kalamata Olives", "Feta Cheese", "Greek Dressing"],
            "Chicken Noodle Soup": ["Chicken", "Egg Noodles", "Carrots", "Celery"],
            "Clam Chowder": ["Clams", "Potatoes", "Bacon", "Cream"]
        },
        "Main": {
            "Grilled Ribeye Steak": ["Ribeye Steak", "Seasoning", "Mashed Potatoes", "Green Beans"],
            "Shrimp Scampi": ["Shrimp", "Garlic", "White Wine", "Linguine"],
            "Vegetable Stir-Fry": ["Assorted Vegetables", "Tofu", "Soy Sauce", "Rice"]
        },
        "Dessert": {
            "Chocolate Lava Cake": ["Chocolate Cake", "Molten Chocolate Center", "Vanilla Ice Cream"],
            "Key Lime Pie": ["Key Lime Filling", "Graham Cracker Crust", "Whipped Cream"],
            "Fruit Sorbet": ["Assorted Fruit Flavors", "Sorbet"]
        }
    }
}


restaurant_A_menu = menu_database["Restaurant A"]
restaurant_B_menu = menu_database["Restaurant B"]


In [15]:
user1={
  "user_id": 1,
  "name": "John",
  "diet": "Vegetarian",
  "allergy": ["Peanuts", "Shellfish"],
  "favorite_cuisine": "Italian"
}


In [27]:
import json

In [24]:
selected_restaurant = "Restaurant A"
selected_menu = menu_database[selected_restaurant]
question = "Based on the user's dietary preferences, allergies, recommend dishes from the selected restaurant menu."

In [28]:

user_context = f"Question: {question}\n\n"
user_context += "Answer the question using the selected restaurant menu and user information below as sources.\n"
user_context += "\n"  
user_context += "###\n"


user_context += f"Selected Restaurant Menu:\n"
user_context += json.dumps(selected_menu, indent=2)
user_context += "\n\n"

user_context += "User Information:\n"
user_context += json.dumps(user1, indent=2)  
user_context += "\n"

user_context += "###\n"

user_prompt = {"role": "user", "content": user_context}


In [30]:
selected_menu

{'Appetizer': {'Bruschetta with Tomato and Basil': ['Baguette',
   'Tomato',
   'Basil',
   'Garlic'],
  'Garlic Butter Shrimp': ['Shrimp', 'Garlic', 'Butter', 'Parsley'],
  'Caprese Salad': ['Tomato', 'Mozzarella', 'Basil', 'Balsamic Glaze']},
 'Salad & Soup': {'Caesar Salad': ['Romaine Lettuce',
   'Croutons',
   'Parmesan Cheese',
   'Caesar Dressing'],
  'Tomato Bisque': ['Tomatoes', 'Cream', 'Onion', 'Garlic'],
  'French Onion Soup': ['Onions', 'Beef Broth', 'Cheese', 'Bread']},
 'Main': {'Grilled Salmon with Lemon Butter': ['Salmon',
   'Lemon',
   'Butter',
   'Asparagus'],
  'Pasta Alfredo with Chicken': ['Pasta',
   'Chicken',
   'Cream Sauce',
   'Parmesan Cheese'],
  'Filet Mignon with Mushroom Sauce': ['Beef',
   'Mushroom Sauce',
   'Potatoes',
   'Green Beans']},
 'Dessert': {'Tiramisu': ['Ladyfingers',
   'Coffee',
   'Mascarpone Cheese',
   'Cocoa Powder'],
  'Chocolate Fondue': ['Chocolate', 'Strawberries', 'Banana', 'Marshmallows'],
  'New York Cheesecake': ['Cream Ch

In [31]:
system_instruction = """
Generate a list of recommended dishes from the selected restaurant menu for the user, considering the following criteria:

1. Dishes that align with the user's dietary preferences and do not contain any of their allergens. These safe dishes form the primary recommendations as they are safe for the user to consume.

2. Within the safe dishes, identify those that also match the user's favorite cuisine preference. Present these dishes in a separate section titled 'Favorites', emphasizing them as they are likely the user's preferred choices.

3. Additionally, identify and list any dishes that might contain allergens as specified by the user. Place these dishes in a section titled 'Dishes with Allergen Warning', clearly indicating the need for the user to avoid them due to potential allergic reactions.

Please structure the recommendations as follows:

- Begin with a section titled 'Safe and Recommended Dishes', listing all dishes that align with the user's dietary preferences and are free of their allergens.
- Follow with a section titled 'Favorites', highlighting dishes from the 'Safe and Recommended Dishes' that also align with the user's favorite cuisine.
- Conclude with a section titled 'Dishes with Allergen Warning', listing dishes that may contain any of the allergens specified by the user.

Ensure the recommendations are clear, concise, and easy to read, following a consistent format throughout the response.
"""
system_prompt = {"role": "system", "content": system_instruction}

In [36]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=[system_prompt, user_prompt],
  temperature=0,
  max_tokens=1000,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response_message = response["choices"][0]["message"]["content"]  

In [37]:
print(response_message )

Based on your dietary preferences and allergies, here are some recommended dishes from the selected restaurant menu:

Safe and Recommended Dishes:
- Bruschetta with Tomato and Basil: Baguette, Tomato, Basil, Garlic
- Caprese Salad: Tomato, Mozzarella, Basil, Balsamic Glaze
- Caesar Salad: Romaine Lettuce, Croutons, Parmesan Cheese, Caesar Dressing

Favorites:
- Bruschetta with Tomato and Basil: Baguette, Tomato, Basil, Garlic
- Caprese Salad: Tomato, Mozzarella, Basil, Balsamic Glaze

Dishes with Allergen Warning:
- Garlic Butter Shrimp: Shrimp, Garlic, Butter, Parsley

Please note that the Garlic Butter Shrimp contains shellfish, which is one of your allergens. It is important to avoid this dish to prevent any allergic reactions.

Enjoy your meal!
